In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

/mydrive


In [4]:
!ls

drive  sample_data


In [5]:
%cd ..

/


In [6]:
!ls

bin	 dev   lib32  mydrive			python-apt  srv    usr
boot	 etc   lib64  NGC-DL-CONTAINER-LICENSE	root	    sys    var
content  home  media  opt			run	    tmp
datalab  lib   mnt    proc			sbin	    tools


In [7]:
%cd /content/drive/MyDrive/Colab Notebooks/Netflix_Movie_Recommendation

/content/drive/MyDrive/Colab Notebooks/Netflix_Movie_Recommendation


In [11]:
!pip install stellargraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached stellargraph-1.2.1-py3-none-any.whl (435 kB)


In [12]:
import matplotlib.pyplot as plt
from math import isclose
from sklearn.decomposition import PCA
from sklearn import preprocessing, feature_extraction, model_selection
import os
import networkx as nx
import numpy as np
import pandas as pd
from stellargraph import StellarGraph, datasets
from stellargraph.data import EdgeSplitter
from collections import Counter
import multiprocessing
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split

%matplotlib inline

In [ ]:
!ls

In [ ]:
!unzip data_folder/archive.zip

In [ ]:
start = datetime.now()
if not os.path.isfile('data.csv'):
    # Creating a file 'data.csv' before reading it
    # Read all the files in netflix and store them in one big file('data.csv')
    # We re reading from each of the four files and appendig each rating to a global file 'train.csv'
    data = open('data.csv', mode='w')
    
    row = list()
    files=['data_folder/combined_data_1.txt','data_folder/combined_data_2.txt', 
           'data_folder/combined_data_3.txt', 'data_folder/combined_data_4.txt']

    for file in files:

        print("Reading ratings from {}...".format(file))

        with open(file) as f:
            for line in f: 
                del row[:] # We might not have to do this.
                line = line.strip()
                if line.endswith(':'):
                    # All below are ratings for this movie, until another movie appears.
                    movie_id = line.replace(':', '')
                else:
                    row = [x for x in line.split(',')]
                    row.insert(0, movie_id)
                    data.write(','.join(row))
                    data.write('\n')
        print("Done.\n")

    data.close()

print('Time taken :', datetime.now() - start)

In [13]:
print("creating the dataframe from data.csv file..")
df = pd.read_csv('data.csv', sep=',', names=['movie', 'user','rating','date'])
df.date = pd.to_datetime(df.date)
print('Done.\n')

# we are arranging the ratings according to time.
print('Sorting the dataframe by date..')
df.sort_values(by='date', inplace=True)
print('Done..')

creating the dataframe from data.csv file..
Done.

Sorting the dataframe by date..
Done..


In [14]:
start_user = 6
end_user   = 100000
mask = (df['user'] >= start_user) & (df['user'] <= end_user)
df2 = df.loc[mask]

In [17]:
df2

,movie,user,rating,date
0,11405,28966,4,1999-12-30
1,15532,28966,1,1999-12-30
2,8903,28966,1,1999-12-30
3,10809,1086,4,1999-12-31
4,829,1086,3,1999-12-31
...,...,...,...,...
3737134,5939,62030,1,2005-12-31
3737135,2873,23421,3,2005-12-31
3737136,6615,23421,5,2005-12-31
3737137,9745,3321,2,2005-12-31


In [16]:
df2 = df2.reset_index(drop=True)

In [18]:
df2_explicit=df2[['movie','user','rating']]
df2_explicit['movie'] = 'm' + df2_explicit['movie'].astype(str)
df2_explicit['user'] = 'u' + df2_explicit['user'].astype(str)
df2_explicit

<ipython-input-18-f05ea4914c6f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_explicit['movie'] = 'm' + df2_explicit['movie'].astype(str)
<ipython-input-18-f05ea4914c6f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_explicit['user'] = 'u' + df2_explicit['user'].astype(str)


,movie,user,rating
0,m11405,u28966,4
1,m15532,u28966,1
2,m8903,u28966,1
3,m10809,u1086,4
4,m829,u1086,3
...,...,...,...
3737134,m5939,u62030,1
3737135,m2873,u23421,3
3737136,m6615,u23421,5
3737137,m9745,u3321,2


In [19]:
df2_implicit=df2[['movie','user']]
df2_implicit['movie'] = 'm' + df2_implicit['movie'].astype(str)
df2_implicit['user'] = 'u' + df2_implicit['user'].astype(str)
df2_implicit.columns = ['source', 'target']

<ipython-input-19-094e0f3edad7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_implicit['movie'] = 'm' + df2_implicit['movie'].astype(str)
<ipython-input-19-094e0f3edad7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_implicit['user'] = 'u' + df2_implicit['user'].astype(str)


In [20]:
df2_implicit

,source,target
0,m11405,u28966
1,m15532,u28966
2,m8903,u28966
3,m10809,u1086
4,m829,u1086
...,...,...
3737134,m5939,u62030
3737135,m2873,u23421
3737136,m6615,u23421
3737137,m9745,u3321


In [21]:
G = StellarGraph(edges=df2_implicit)
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 35770, Edges: 3737139

 Node types:
  default: [35770]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [3737139]
        Weights: all 1 (default)
        Features: none


In [22]:
# Define an edge splitter on the original graph:
edge_splitter_test = EdgeSplitter(G)

# Randomly sample a fraction p=0.2 of all positive links, and same number of negative links, from graph, and obtain the
# reduced graph graph_test with the sampled links removed:
graph_test, examples_test, labels_test = edge_splitter_test.train_test_split(
    p=0.2, method="global", keep_connected = True
)

print(graph_test.info())

** Sampled 747427 positive and 747427 negative edges. **
StellarGraph: Undirected multigraph
 Nodes: 35770, Edges: 2989712

 Node types:
  default: [35770]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [2989712]
        Weights: all 1 (default)
        Features: none


In [23]:
373713*2

747426

In [24]:
len(examples_test)

1494854

In [25]:
3363426-3027084

336342

In [26]:
# Do the same process to compute a training subset from within the test graph
edge_splitter_train = EdgeSplitter(graph_test, G)
graph_train, examples, labels = edge_splitter_train.train_test_split(
    p=0.2, method="global", keep_connected = True
)
(
    examples_train,
    examples_model_selection,
    labels_train,
    labels_model_selection,
) = train_test_split(examples, labels, train_size=0.8, test_size=0.2)

print(graph_train.info())

** Sampled 597942 positive and 597942 negative edges. **
StellarGraph: Undirected multigraph
 Nodes: 35770, Edges: 2391770

 Node types:
  default: [35770]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [2391770]
        Weights: all 1 (default)
        Features: none


In [27]:
examples_model_selection

array([['m187', 'u70328'],
       ['u39831', 'u28213'],
       ['m6386', 'u83805'],
       ...,
       ['m6187', 'u37451'],
       ['m8954', 'u90742'],
       ['m8117', 'u42345']], dtype=object)

In [28]:
labels_model_selection

array([1, 0, 1, ..., 0, 1, 1])

In [29]:
336342*2

672684

In [30]:
len(examples)

1195884

In [31]:
len(examples_train)

956707

#Deep Walk

In [32]:
dimensions = 32
num_walks = 10
walk_length = 40
window_size = 5
num_iter = 1
workers = multiprocessing.cpu_count()

In [33]:
from gensim.models import Word2Vec
from stellargraph.data import UniformRandomWalk

def deepWalk_embedding_train(graph, name):
    rw = UniformRandomWalk(graph)
    walks = rw.run(graph.nodes(), n=num_walks, length=walk_length)
    print(f"Number of random walks for '{name}': {len(walks)}")

    model = Word2Vec(
        walks,
        size=dimensions,
        window=window_size,
        min_count=0,
        sg=1,
        workers=workers,
        iter=num_iter,
    )

    model.save("deepWalk_embedding_train.model")
    def get_embedding(u):
        return model.wv[u]

    return get_embedding

def deepWalk_embedding_test(graph, name):
    rw = UniformRandomWalk(graph)
    walks = rw.run(graph.nodes(), n=num_walks, length=walk_length)
    print(f"Number of random walks for '{name}': {len(walks)}")

    model = Word2Vec(
        walks,
        size=dimensions,
        window=window_size,
        min_count=0,
        sg=1,
        workers=workers,
        iter=num_iter,
    )

    model.save("deepWalk_embedding_test.model")
    def get_embedding(u):
        return model.wv[u]

    return get_embedding

In [34]:
embedding_train_deepW = deepWalk_embedding_train(graph_train, "Train Graph")

Number of random walks for 'Train Graph': 357700


In [35]:
embedding_train_deepW('u18702')

array([ 0.32066625,  0.20943502,  0.11771876, -0.5377317 ,  0.10620353,
        0.02973612, -0.3471631 ,  0.01199626,  0.04199905, -0.03668234,
       -0.15197623, -0.13650276, -0.10623594,  0.0763692 , -0.0252196 ,
       -0.25288713, -0.66952896,  0.26896286, -0.02152866, -0.10220387,
       -0.558724  , -0.97082484, -0.06930302,  0.1850806 , -0.3388855 ,
       -0.01407279, -0.3689696 ,  0.15004921, -0.04440913, -0.5713346 ,
       -0.01487237, -0.5684083 ], dtype=float32)

In [36]:
embedding_test_deepW = deepWalk_embedding_test(graph_test, "Test Graph")

Number of random walks for 'Test Graph': 357700


# Implicit DW

In [37]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras

model_concat_dw = Sequential()

model_concat_dw.add(Dense(units=64, input_dim=64, kernel_initializer='normal', activation='relu'))

model_concat_dw.add(Dense(units=32, kernel_initializer='normal', activation='tanh'))

model_concat_dw.add(Dense(units=16, kernel_initializer='normal', activation='tanh'))

model_concat_dw.add(Dense(units=8, kernel_initializer='normal', activation='tanh'))

model_concat_dw.add(Dense(units=4, kernel_initializer='normal', activation='tanh'))

model_concat_dw.add(Dense(1, kernel_initializer='normal'))

opt = keras.optimizers.Adam(learning_rate=0.01)

model_concat_dw.compile(loss='mean_squared_error', optimizer='adam')

In [38]:
def train_model_concatdw(
    link_examples, link_labels, get_embedding, binary_operator
):
    link_features = link_examples_to_features(
        link_examples, get_embedding, binary_operator
    )
    model_concat_dw.fit(np.array(link_features), np.array(link_labels), batch_size = 50, epochs = 50, verbose=1)
    return model_concat_dw

In [39]:
def operator_concat(u, v):
    return np.concatenate((u, v))
def link_examples_to_features(link_examples, transform_node, binary_operator):
    return [
        binary_operator(transform_node(src), transform_node(dst))
        for src, dst in link_examples
    ]

In [40]:
clf_dw_concat = train_model_concatdw(
        examples_train, labels_train, embedding_train_deepW, operator_concat
    )

Epoch 1/50
19135/19135 [==============================] - 42s 2ms/step - loss: 0.0367
Epoch 2/50
19135/19135 [==============================] - 39s 2ms/step - loss: 0.0321
Epoch 3/50
19135/19135 [==============================] - 40s 2ms/step - loss: 0.0315
Epoch 4/50
19135/19135 [==============================] - 39s 2ms/step - loss: 0.0311
Epoch 5/50
19135/19135 [==============================] - 40s 2ms/step - loss: 0.0306
Epoch 6/50
19135/19135 [==============================] - 41s 2ms/step - loss: 0.0303
Epoch 7/50
19135/19135 [==============================] - 41s 2ms/step - loss: 0.0301
Epoch 8/50
19135/19135 [==============================] - 41s 2ms/step - loss: 0.0299
Epoch 9/50
19135/19135 [==============================] - 43s 2ms/step - loss: 0.0298
Epoch 10/50
19135/19135 [==============================] - 39s 2ms/step - loss: 0.0296
Epoch 11/50
19135/19135 [==============================] - 40s 2ms/step - loss: 0.0295
Epoch 12/50
19135/19135 [===========================

In [41]:
clf_dw_concat.save('clf_dw_concat.model')
# from tensorflow import keras
# model = keras.models.load_model('clf_dw_concat.model')

In [42]:
link_features_selection_dw = link_examples_to_features(
        examples_model_selection, embedding_train_deepW, operator_concat)

In [43]:
predicted_dw = clf_dw_concat.predict(np.array(link_features_selection_dw))

7475/7475 [==============================] - 10s 1ms/step


In [44]:
print(predicted_dw)

[[0.98886985]
 [0.00173074]
 [0.9897951 ]
 ...
 [0.19212204]
 [0.9894196 ]
 [0.8924854 ]]


In [45]:
predicted_dw=predicted_dw.flatten()

In [46]:
loss_dw_con=np.sum((labels_model_selection-predicted_dw)**2)/len(predicted_dw)
loss_dw_con

0.029017725830540318

In [47]:
def get_label(i):
  if i>=0.5:
    i=1
  else:
    i=0
  return i

In [48]:
predicted_label_concat_dw=np.array([get_label(i) for i in predicted_dw])
accuracy_concat_dw=1-np.sum(abs(labels_model_selection-predicted_label_concat_dw))/len(predicted_dw)
accuracy_concat_dw

0.9623793257712907

# Test our DeepWalk model with implicit feedback

In [49]:
movie_unique_dw=df2_implicit.source.unique()
array_true_im_testdw=[]
for i in range(len(labels_test)):
  if labels_test[i]==1:
    array_true_im_testdw.append(examples_test[i])

from collections import defaultdict
import json
def create_user_movie_dict_dw(array_true):

    # if not os.path.isfile('user_movie_train.json'):
    user_movie = defaultdict(list)
    for i in range(len(array_true)):
        user_movie[array_true[i][1]].append(array_true[i][0])
        # my_json = json.dumps(user_movie)
        # f = open("user_movie_train.json","w")
        # f.write(my_json)
        # f.close()
    # else:
    #     print("opening json file")
    #     with open('user_movie_train.json') as json_file:
    #       user_movie = json.load(json_file)
    # print('user_movie_train.json loaded')
    return user_movie

dict_dw_im_test=create_user_movie_dict_dw(array_true_im_testdw)

array_true_user_im=list(dict_dw_im_test.keys())

import random
test_implicit={}
for i in range(len(array_true_user_im)):
  negative_example=[]
  negative_example.append(dict_dw_im_test[array_true_user_im[i]][0])
  # print(negative_example)
  j=0
  movie_watched=dict_dw_im_test[array_true_user_im[i]]
  while(j<99):
    movie_picked=movie_unique_dw[random.randint(0, 17674)]
    if movie_picked in movie_watched:
      continue
    else:
      negative_example.append(movie_picked)
      j+=1
  test_implicit[array_true_user_im[i]]=negative_example

In [50]:
test_implicit

{'u39568': ['m3925',
  'm2715',
  'm3218',
  'm8166',
  'm15510',
  'm5962',
  'm13311',
  'm8201',
  'm12907',
  'm4025',
  'm7695',
  'm15610',
  'm1409',
  'm10586',
  'm10915',
  'm16990',
  'm7606',
  'm15617',
  'm11945',
  'm959',
  'm6936',
  'm10479',
  'm17611',
  'm6955',
  'm2541',
  'm11129',
  'm406',
  'm979',
  'm12265',
  'm12400',
  'm6784',
  'm6570',
  'm13771',
  'm12891',
  'm4052',
  'm5535',
  'm5405',
  'm11660',
  'm12741',
  'm13556',
  'm15759',
  'm3183',
  'm842',
  'm11996',
  'm11203',
  'm8351',
  'm10589',
  'm6044',
  'm16325',
  'm2735',
  'm999',
  'm6882',
  'm7907',
  'm1073',
  'm6463',
  'm2724',
  'm17019',
  'm753',
  'm7957',
  'm3606',
  'm2563',
  'm4570',
  'm6188',
  'm10868',
  'm17317',
  'm13089',
  'm11599',
  'm17236',
  'm12247',
  'm6740',
  'm17120',
  'm14654',
  'm13174',
  'm6314',
  'm10848',
  'm12261',
  'm2553',
  'm1829',
  'm13670',
  'm14916',
  'm11206',
  'm5178',
  'm15036',
  'm7340',
  'm11498',
  'm7805',
  'm5570'

In [51]:
examples_train

array([['m14296', 'u88118'],
       ['m15339', 'u70392'],
       ['m8325', 'm9594'],
       ...,
       ['m3860', 'u30218'],
       ['u42620', 'm11797'],
       ['m3282', 'u55445']], dtype=object)

In [52]:
b=[['u77970', 'u97877'],
       ['m5760', 'u60561']]
b.append(['m5760', 'u60561'])
print(b)

[['u77970', 'u97877'], ['m5760', 'u60561'], ['m5760', 'u60561']]


In [ ]:
HR=0
ndcg=0
HR_F=0
ndcg_F=0
for i in list(test_implicit.keys()):
  test=[]
  
  negatives=test_implicit[i]
  # print(negatives)
  for j in negatives:
    a=[]
    a.append(i)
    for z in range(1):
      a.append(j)
    #print(a)
    test.append(a)
  #test=np.array(test)
  #print(test)
  test_features=link_examples_to_features(np.array(test), embedding_test_deepW, operator_concat)
  #print(test_features)
  predicted_scores=clf_dw_concat.predict(np.array(test_features))
  predicted_scores=np.array(predicted_scores.flatten())
  predicted_scores=np.abs(predicted_scores)
  #print(predicted_scores)
  #print(type(predicted_scores))
  true_score=predicted_scores[0]
  #print(true_score)
  predicted_scores=np.sort(predicted_scores)
  
  predicted_scores=predicted_scores[::-1]
  #print(predicted_scores)
  #print(predicted_scores[:10])
  if true_score in predicted_scores[:10]:
    HR+=1
    
    index = np.where(predicted_scores[:10] == true_score)
    #print(index)
    ndcg += np.reciprocal(np.log2(index[0][0]+2))
  
  # print(HR)
HR_F=HR/len(list(test_implicit.keys()))
ndcg_F=ndcg/len(list(test_implicit.keys()))


In [3]:
print(HR_F)
print(ndcg_F)

0.48312628451333334
0.4621215123489127


# Explicit Dw

# build model

In [56]:
from tensorflow import keras

model_concat_ex_dw = Sequential()

model_concat_ex_dw.add(Dense(units=64, input_dim=64, kernel_initializer='normal', activation='relu'))

model_concat_ex_dw.add(Dense(units=32, kernel_initializer='normal', activation='tanh'))

model_concat_ex_dw.add(Dense(units=16, kernel_initializer='normal', activation='tanh'))

model_concat_ex_dw.add(Dense(units=8, kernel_initializer='normal', activation='tanh'))

model_concat_ex_dw.add(Dense(units=4, kernel_initializer='normal', activation='tanh'))

model_concat_ex_dw.add(Dense(1, kernel_initializer='normal'))

opt = keras.optimizers.Adam(learning_rate=0.01)

model_concat_ex_dw.compile(loss='mean_squared_error', optimizer='adam')

In [57]:
def train_model_concat_ex_dw(
    link_examples, link_labels, get_embedding, binary_operator
):
    link_features = link_examples_to_features(
        link_examples, get_embedding, binary_operator
    )
    model_concat_ex_dw.fit(np.array(link_features), np.array(link_labels), batch_size = 50, epochs = 40, verbose=1)
    return model_concat_ex_dw

In [58]:
array_true_ex_train=[]
for i in range(len(labels_train)):
  if labels_train[i]==1:
    array_true_ex_train.append(examples_train[i])

from collections import defaultdict
import json
def create_user_movie_dict_df2ex(df):

    # if not os.path.isfile('user_movie_train.json'):
    user_movie = defaultdict(dict)
    for iter, row in df.iterrows():
        
        dict1={}
        dict1=user_movie[row[1]].copy()
        dict1[row[0]]=row[2]
        user_movie[row[1]]=dict1
    #     my_json = json.dumps(user_movie)
    #     f = open("user_movie_train.json","w")
    #     f.write(my_json)
    #     f.close()
    # else:
    #     print("opening json file")
    #     with open('user_movie_train.json') as json_file:
    #       user_movie = json.load(json_file)
    # print('user_movie_train.json loaded')
    return user_movie

df2_explicit_dict=create_user_movie_dict_df2ex(df2_explicit)

ratings_train=[]
for i in range(len(array_true_ex_train)):
  movie=array_true_ex_train[i][0]
  user=array_true_ex_train[i][1]
  ratings_train.append(df2_explicit_dict[user][movie])
len(ratings_train)

array_true_ex_select=[]
for i in range(len(labels_model_selection)):
  if labels_model_selection[i]==1:
    array_true_ex_select.append(examples_model_selection[i])

ratings_selection=[]
for i in range(len(array_true_ex_select)):
  movie=array_true_ex_select[i][0]
  user=array_true_ex_select[i][1]
  ratings_selection.append(df2_explicit_dict[user][movie])
len(ratings_selection)

119661

In [59]:
clf_concat_ex_dw = train_model_concat_ex_dw(
        array_true_ex_train, ratings_train, embedding_train_deepW, operator_concat
    )

Epoch 1/40
9566/9566 [==============================] - 21s 2ms/step - loss: 1.4644
Epoch 2/40
9566/9566 [==============================] - 20s 2ms/step - loss: 0.9975
Epoch 3/40
9566/9566 [==============================] - 19s 2ms/step - loss: 0.9773
Epoch 4/40
9566/9566 [==============================] - 19s 2ms/step - loss: 0.9693
Epoch 5/40
9566/9566 [==============================] - 20s 2ms/step - loss: 0.9649
Epoch 6/40
9566/9566 [==============================] - 21s 2ms/step - loss: 0.9616
Epoch 7/40
9566/9566 [==============================] - 20s 2ms/step - loss: 0.9586
Epoch 8/40
9566/9566 [==============================] - 21s 2ms/step - loss: 0.9566
Epoch 9/40
9566/9566 [==============================] - 20s 2ms/step - loss: 0.9547
Epoch 10/40
9566/9566 [==============================] - 20s 2ms/step - loss: 0.9530
Epoch 11/40
9566/9566 [==============================] - 20s 2ms/step - loss: 0.9516
Epoch 12/40
9566/9566 [==============================] - 20s 2ms/step - lo

In [60]:
clf_concat_ex_dw.save('clf_concat_ex_dw.model')
# from tensorflow import keras
# model = keras.models.load_model('model_concat_ex_dw.model')

In [61]:
array_true_ex_test=[]
for i in range(len(labels_test)):
  if labels_test[i]==1:
    array_true_ex_test.append(examples_test[i])

ratings_test=[]
for i in range(len(array_true_ex_test)):
  movie=array_true_ex_test[i][0]
  user=array_true_ex_test[i][1]
  ratings_test.append(df2_explicit_dict[user][movie])
len(ratings_test)

link_features_test_ex = link_examples_to_features(
        array_true_ex_test, embedding_test_deepW, operator_concat)
predicted_concat_test_ex = clf_concat_ex_dw.predict(np.array(link_features_test_ex))
predicted_concat_test_ex=predicted_concat_test_ex.flatten()

loss_concat_test_ex=np.sum((ratings_test-predicted_concat_test_ex)**2)/len(predicted_concat_test_ex)
loss_concat_test_ex

RMSE=np.sqrt(np.sum((np.array(ratings_test)-predicted_concat_test_ex)**2)/len(predicted_concat_test_ex))
RMSE

MAPE=np.sum(np.abs((np.array(ratings_test)-predicted_concat_test_ex))/ratings_test)/len(predicted_concat_test_ex)
MAPE

23358/23358 [==============================] - 28s 1ms/step


0.40126858761523726

In [62]:
loss_concat_test_ex

1.4459614105866245

In [63]:
RMSE

1.2024813556087364

In [64]:
MAPE

0.40126858761523726

# Node2Vec

In [65]:
p = 1.0
q = 0.5
dimensions = 32
num_walks = 10
walk_length = 40
window_size = 5
num_iter = 1
workers = multiprocessing.cpu_count()

In [66]:
from stellargraph.data import BiasedRandomWalk
from gensim.models import Word2Vec


def node2vec_embedding_train(graph, name):
    rw = BiasedRandomWalk(graph)
    walks = rw.run(graph.nodes(), n=num_walks, length=walk_length, p=p, q=q)
    print(f"Number of random walks for '{name}': {len(walks)}")

    model = Word2Vec(
        walks,
        size=dimensions,
        window=window_size,
        min_count=0,
        sg=1,
        workers=workers,
        iter=num_iter,
    )

    model.save("node2vec_embedding_train.model")
    def get_embedding(u):
        return model.wv[u]

    return get_embedding

def node2vec_embedding_test(graph, name):
    rw = BiasedRandomWalk(graph)
    walks = rw.run(graph.nodes(), n=num_walks, length=walk_length, p=p, q=q)
    print(f"Number of random walks for '{name}': {len(walks)}")

    model = Word2Vec(
        walks,
        size=dimensions,
        window=window_size,
        min_count=0,
        sg=1,
        workers=workers,
        iter=num_iter,
    )

    model.save("node2vec_embedding_test.model")
    def get_embedding(u):
        return model.wv[u]

    return get_embedding

In [67]:
embedding_train = node2vec_embedding_train(graph_train, "Train Graph")

Number of random walks for 'Train Graph': 357700


In [68]:
embedding_test = node2vec_embedding_test(graph_test, "Test Graph")

Number of random walks for 'Test Graph': 357700


In [69]:
embedding_train('u54902')

array([ 0.23925376,  0.40896732,  0.35928875, -0.26472804,  0.3561669 ,
       -0.0639363 , -0.35883102, -0.46279582, -0.02292539,  0.22478591,
       -0.18858702, -0.26382217,  0.21360822,  0.08878147, -0.3409715 ,
       -0.2615404 , -0.14342514, -0.0294617 ,  0.09424245, -0.43551356,
       -0.16138169, -0.8301645 ,  0.11483356,  0.07110158, -0.8059631 ,
        0.01564322, -0.33640248, -0.04425808, -0.32322735, -0.67067045,
       -0.00175937, -0.5483301 ], dtype=float32)

# Node2Vec implicit

In [70]:
def operator_l1(u, v):
    return np.abs(u - v)

def operator_concat(u, v):
    return np.concatenate((u, v))

def link_examples_to_features(link_examples, transform_node, binary_operator):
    return [
        binary_operator(transform_node(src), transform_node(dst))
        for src, dst in link_examples
    ]

In [71]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras

model_concat = Sequential()

model_concat.add(Dense(units=64, input_dim=64, kernel_initializer='normal', activation='relu'))

model_concat.add(Dense(units=32, kernel_initializer='normal', activation='tanh'))

model_concat.add(Dense(units=16, kernel_initializer='normal', activation='tanh'))

model_concat.add(Dense(units=8, kernel_initializer='normal', activation='tanh'))

model_concat.add(Dense(units=4, kernel_initializer='normal', activation='tanh'))

model_concat.add(Dense(1, kernel_initializer='normal'))

model_concat.compile(loss='mean_squared_error', optimizer='adam')

In [72]:
model_l1 = Sequential()

model_l1.add(Dense(units=64, input_dim=32, kernel_initializer='normal', activation='relu'))

model_l1.add(Dense(units=32, kernel_initializer='normal', activation='tanh'))

model_l1.add(Dense(units=16, kernel_initializer='normal', activation='tanh'))

model_l1.add(Dense(units=8, kernel_initializer='normal', activation='tanh'))

model_l1.add(Dense(units=4, kernel_initializer='normal', activation='tanh'))

model_l1.add(Dense(1, kernel_initializer='normal'))

model_l1.compile(loss='mean_squared_error', optimizer='adam')

In [73]:
def train_model_l1(
    link_examples, link_labels, get_embedding, binary_operator
):
    print('1')
    link_features = link_examples_to_features(
        link_examples, get_embedding, binary_operator
    )
    print(link_features[0])
    print(np.array(link_features)[0])
    print(link_labels.shape)
    print('2')
    model_l1.fit(np.array(link_features), np.array(link_labels), batch_size = 50, epochs = 20, verbose=1)
    print('3')
    return model_l1

def train_model_concat(
    link_examples, link_labels, get_embedding, binary_operator
):
    link_features = link_examples_to_features(
        link_examples, get_embedding, binary_operator
    )
    model_concat.fit(np.array(link_features), np.array(link_labels), batch_size = 50, epochs = 20, verbose=1)
    return model_concat

In [74]:
len(examples_train)

956707

In [75]:
labels_train

array([1, 1, 0, ..., 1, 0, 1])

In [76]:
examples_train[:15]

array([['m14296', 'u88118'],
       ['m15339', 'u70392'],
       ['m8325', 'm9594'],
       ['m4892', 'm8756'],
       ['m9028', 'u45538'],
       ['m14283', 'm16492'],
       ['u39606', 'u14924'],
       ['m8905', 'u80290'],
       ['m4640', 'u31985'],
       ['m14171', 'm7800'],
       ['m1440', 'u25075'],
       ['m8832', 'u98977'],
       ['m7617', 'u2807'],
       ['u97046', 'u65078'],
       ['m12845', 'm15985']], dtype=object)

# Operator L1

In [77]:
clf_l1 = train_model_l1(
        examples_train, labels_train, embedding_train, operator_l1
    )

1
[0.4116749  0.17243585 0.24843372 0.3366567  0.29452354 0.52074116
 0.5503845  0.03506192 0.22441533 0.07882293 0.33449212 0.0445069
 0.19600487 0.44226813 0.55260766 0.26600304 0.4072196  0.03820133
 0.01790228 0.17435056 0.47702584 0.12407637 0.45112982 0.03233886
 0.26577783 0.09833881 0.3821273  0.21264076 0.12191042 0.0495812
 0.16194168 0.22479443]
[0.4116749  0.17243585 0.24843372 0.3366567  0.29452354 0.52074116
 0.5503845  0.03506192 0.22441533 0.07882293 0.33449212 0.0445069
 0.19600487 0.44226813 0.55260766 0.26600304 0.4072196  0.03820133
 0.01790228 0.17435056 0.47702584 0.12407637 0.45112982 0.03233886
 0.26577783 0.09833881 0.3821273  0.21264076 0.12191042 0.0495812
 0.16194168 0.22479443]
(956707,)
2
Epoch 1/20
19135/19135 [==============================] - 38s 2ms/step - loss: 0.0746
Epoch 2/20
19135/19135 [==============================] - 37s 2ms/step - loss: 0.0660
Epoch 3/20
19135/19135 [==============================] - 38s 2ms/step - loss: 0.0639
Epoch 4/20
191

In [78]:
clf_l1.save('clf_l1.model')
# from tensorflow import keras
# model = keras.models.load_model('clf_l1.model')

In [79]:
link_features_test1 = link_examples_to_features(
        examples_model_selection, embedding_train, operator_l1)

In [80]:
type(link_features_test1[0])

numpy.ndarray

In [81]:
labels_model_selection

array([1, 0, 1, ..., 0, 1, 1])

In [82]:
predicted = clf_l1.predict(np.array(link_features_test1))

7475/7475 [==============================] - 9s 1ms/step


In [83]:
predicted=predicted.flatten()

In [84]:
len(predicted)

239177

In [85]:
len(labels_model_selection)

239177

In [86]:
loss_l1=np.sum((labels_model_selection-predicted)**2)/len(predicted)
loss_l1

0.059104245894771305

In [87]:
def get_label(i):
  if i>=0.5:
    i=1
  else:
    i=0
  return i

In [88]:
predicted_label=np.array([get_label(i) for i in predicted])

In [89]:
predicted_label

array([1, 0, 1, ..., 0, 1, 1])

In [90]:
labels_model_selection

array([1, 0, 1, ..., 0, 1, 1])

In [91]:
accuracy_l1=1-np.sum(abs(labels_model_selection-predicted_label))/len(predicted)
accuracy_l1

0.9223671172395339

# Operator Concat

In [92]:
clf_concat = train_model_concat(
        examples_train, labels_train, embedding_train, operator_concat
    )

Epoch 1/20
19135/19135 [==============================] - 39s 2ms/step - loss: 0.0362
Epoch 2/20
19135/19135 [==============================] - 38s 2ms/step - loss: 0.0310
Epoch 3/20
19135/19135 [==============================] - 37s 2ms/step - loss: 0.0304
Epoch 4/20
19135/19135 [==============================] - 39s 2ms/step - loss: 0.0299
Epoch 5/20
19135/19135 [==============================] - 39s 2ms/step - loss: 0.0296
Epoch 6/20
19135/19135 [==============================] - 38s 2ms/step - loss: 0.0294
Epoch 7/20
19135/19135 [==============================] - 39s 2ms/step - loss: 0.0292
Epoch 8/20
19135/19135 [==============================] - 40s 2ms/step - loss: 0.0290
Epoch 9/20
19135/19135 [==============================] - 39s 2ms/step - loss: 0.0289
Epoch 10/20
19135/19135 [==============================] - 39s 2ms/step - loss: 0.0288
Epoch 11/20
19135/19135 [==============================] - 39s 2ms/step - loss: 0.0287
Epoch 12/20
19135/19135 [===========================

In [93]:
clf_concat.save('clf_concat.model')
# from tensorflow import keras
# clf_concat = keras.models.load_model('clf_concat.model')

In [94]:
link_features_test2 = link_examples_to_features(
        examples_model_selection, embedding_train, operator_concat)

In [95]:
predicted_concat = clf_concat.predict(np.array(link_features_test2))
predicted_concat=predicted_concat.flatten()

7475/7475 [==============================] - 9s 1ms/step


In [96]:
loss_concat=np.sum((labels_model_selection-predicted_concat)**2)/len(predicted_concat)
loss_concat

0.029102988450533673

In [97]:
labels_model_selection[:10]

array([1, 0, 1, 1, 1, 0, 0, 0, 0, 0])

In [98]:
predicted_concat[:10]

array([ 9.9859428e-01, -6.3061714e-05,  9.9928582e-01,  9.2774773e-01,
        9.8056495e-01,  9.6511841e-04,  1.4388561e-03,  2.8230494e-01,
        5.3220987e-04,  1.6236305e-04], dtype=float32)

In [99]:
predicted_label_concat=np.array([get_label(i) for i in predicted_concat])
accuracy_concat=1-np.sum(abs(labels_model_selection-predicted_label_concat))/len(predicted_concat)
accuracy_concat

0.961756356171371

# test our model, node2vec implicit

In [100]:
movie_unique=df2_implicit.source.unique()
array_true_im_test=[]
for i in range(len(labels_test)):
  if labels_test[i]==1:
    array_true_im_test.append(examples_test[i])

from collections import defaultdict
import json
def create_user_movie_dict_array_true(array_true):

    # if not os.path.isfile('user_movie_train.json'):
    user_movie = defaultdict(list)
    for i in range(len(array_true)):
        user_movie[array_true[i][1]].append(array_true[i][0])
        # my_json = json.dumps(user_movie)
        # f = open("user_movie_train.json","w")
        # f.write(my_json)
        # f.close()
    # else:
    #     print("opening json file")
    #     with open('user_movie_train.json') as json_file:
    #       user_movie = json.load(json_file)
    # print('user_movie_train.json loaded')
    return user_movie

dict_array_true_im_test=create_user_movie_dict_array_true(array_true_im_test)

array_true_user_im=list(dict_array_true_im_test.keys())

import random
test_implicit={}
for i in range(len(array_true_user_im)):
  negative_example=[]
  negative_example.append(dict_array_true_im_test[array_true_user_im[i]][0])
  # print(negative_example)
  j=0
  movie_watched=dict_array_true_im_test[array_true_user_im[i]]
  while(j<99):
    movie_picked=movie_unique[random.randint(0, 17674)]
    if movie_picked in movie_watched:
      continue
    else:
      negative_example.append(movie_picked)
      j+=1
  test_implicit[array_true_user_im[i]]=negative_example

In [101]:
dict_array_true_im_test['u42128']

['m2698',
 'm8569',
 'm16265',
 'm1722',
 'm9003',
 'm5924',
 'm13007',
 'm7430',
 'm6756',
 'm4490',
 'm4883',
 'm17261',
 'm7330',
 'm4033',
 'm11701',
 'm2808',
 'm3787',
 'm2916',
 'm13974',
 'm15156',
 'm9593',
 'm5277',
 'm5087',
 'm459',
 'm7917',
 'm5187',
 'm831',
 'm17097',
 'm12442',
 'm13423',
 'm9032',
 'm11209',
 'm6076',
 'm11186',
 'm15296',
 'm6206',
 'm9350',
 'm4577',
 'm8317',
 'm6302',
 'm10013',
 'm1901',
 'm13023',
 'm10255',
 'm5997',
 'm15758',
 'm10504',
 'm3825',
 'm14233',
 'm13546',
 'm14480',
 'm16260',
 'm12527',
 'm1700',
 'm14660',
 'm5788',
 'm9960',
 'm10889',
 'm12497',
 'm550',
 'm8904',
 'm17474',
 'm30',
 'm12671',
 'm7047',
 'm14943',
 'm11446',
 'm11271',
 'm191',
 'm12425',
 'm9617',
 'm12359',
 'm10947',
 'm14467',
 'm5414',
 'm6255',
 'm918',
 'm6366',
 'm13959',
 'm6574',
 'm16344',
 'm15062',
 'm4267',
 'm11717',
 'm10921',
 'm6231',
 'm15788',
 'm16008',
 'm13129',
 'm11198',
 'm6974',
 'm6158',
 'm10820',
 'm15124',
 'm8989',
 'm14928',
 

In [102]:
len(test_implicit['u42128'])

100

# L1

In [103]:
HR=0
ndcg=0
for i in list(test_implicit.keys()):
  test=[]
  
  negatives=test_implicit[i]
  # print(negatives)
  for j in negatives:
    a=[]
    a.append(i)
    for z in range(1):
      a.append(j)
    #print(a)
    test.append(a)
  #test=np.array(test)
  #print(test)
  test_features=link_examples_to_features(np.array(test), embedding_test_deepW, operator_l1)
  #print(test_features)
  predicted_scores=clf_l1.predict(np.array(test_features))
  predicted_scores=np.array(predicted_scores.flatten())
  predicted_scores=np.abs(predicted_scores)
  #print(predicted_scores)
  #print(type(predicted_scores))
  true_score=predicted_scores[0]
  #print(true_score)
  predicted_scores=np.sort(predicted_scores)
  
  predicted_scores=predicted_scores[::-1]
  #print(predicted_scores)
  #print(predicted_scores[:10])
  if true_score in predicted_scores[:10]:
    HR+=1
    
    index = np.where(predicted_scores[:10] == true_score)
    #print(index)
    ndcg += np.reciprocal(np.log2(index[0][0]+2))
  
  # print(HR)
HR_F=HR/len(list(test_implicit.keys()))
ndcg_F=ndcg/len(list(test_implicit.keys()))
print(HR_F)
print(ndcg_F)

串流輸出內容已截斷至最後 5000 行。
4/4 [==============================] - 0s 2ms/step
0.40456852791878173
0.20708756915196785


In [104]:
print(HR_F)
print(ndcg_F)

0.40456852791878173
0.20708756915196785


# Concat

In [105]:
HR=0
ndcg=0
for i in list(test_implicit.keys()):
  test=[]
  
  negatives=test_implicit[i]
  # print(negatives)
  for j in negatives:
    a=[]
    a.append(i)
    for z in range(1):
      a.append(j)
    #print(a)
    test.append(a)
  #test=np.array(test)
  #print(test)
  test_features=link_examples_to_features(np.array(test), embedding_test_deepW, operator_concat)
  #print(test_features)
  predicted_scores=clf_concat.predict(np.array(test_features))
  predicted_scores=np.array(predicted_scores.flatten())
  predicted_scores=np.abs(predicted_scores)
  #print(predicted_scores)
  #print(type(predicted_scores))
  true_score=predicted_scores[0]
  #print(true_score)
  predicted_scores=np.sort(predicted_scores)
  
  predicted_scores=predicted_scores[::-1]
  #print(predicted_scores)
  #print(predicted_scores[:10])
  if true_score in predicted_scores[:10]:
    HR+=1
    
    index = np.where(predicted_scores[:10] == true_score)
    #print(index)
    ndcg += np.reciprocal(np.log2(index[0][0]+2))
  
  # print(HR)
HR_F=HR/len(list(test_implicit.keys()))
ndcg_F=ndcg/len(list(test_implicit.keys()))
print(HR_F)
print(ndcg_F)

串流輸出內容已截斷至最後 5000 行。
4/4 [==============================] - 0s 3ms/step
0.5145516074450085
0.33373923137959716


In [106]:
print(HR_F)
print(ndcg_F)

0.5145516074450085
0.33373923137959716


In [107]:
0.49412694827196746
0.4720186086826375

0.4720186086826375

# explicit feedback

# build model

In [108]:
from tensorflow import keras

model_concat_ex = Sequential()

model_concat_ex.add(Dense(units=64, input_dim=64, kernel_initializer='normal', activation='relu'))

model_concat_ex.add(Dense(units=32, kernel_initializer='normal', activation='tanh'))

model_concat_ex.add(Dense(units=16, kernel_initializer='normal', activation='tanh'))

model_concat_ex.add(Dense(units=8, kernel_initializer='normal', activation='tanh'))

model_concat_ex.add(Dense(units=4, kernel_initializer='normal', activation='tanh'))

model_concat_ex.add(Dense(1, kernel_initializer='normal'))

opt = keras.optimizers.Adam(learning_rate=0.01)

model_concat_ex.compile(loss='mean_squared_error', optimizer='adam')

In [109]:
def train_model_concat_ex(
    link_examples, link_labels, get_embedding, binary_operator
):
    link_features = link_examples_to_features(
        link_examples, get_embedding, binary_operator
    )
    model_concat.fit(np.array(link_features), np.array(link_labels), batch_size = 50, epochs = 40, verbose=1)
    return model_concat

In [110]:
from tensorflow import keras

model_l1_ex = Sequential()

model_l1_ex.add(Dense(units=64, input_dim=32, kernel_initializer='normal', activation='relu'))

model_l1_ex.add(Dense(units=32, kernel_initializer='normal', activation='tanh'))

model_l1_ex.add(Dense(units=16, kernel_initializer='normal', activation='tanh'))

model_l1_ex.add(Dense(units=8, kernel_initializer='normal', activation='tanh'))

model_l1_ex.add(Dense(units=4, kernel_initializer='normal', activation='tanh'))

model_l1_ex.add(Dense(1, kernel_initializer='normal'))

opt = keras.optimizers.Adam(learning_rate=0.01)

model_l1_ex.compile(loss='mean_squared_error', optimizer='adam')

In [111]:
def train_model_l1_ex(
    link_examples, link_labels, get_embedding, binary_operator
):
    link_features = link_examples_to_features(
        link_examples, get_embedding, binary_operator
    )
    model_l1_ex.fit(np.array(link_features), np.array(link_labels), batch_size = 50, epochs = 40, verbose=1)
    return model_l1_ex

#Prepare training dataset

In [112]:
len(examples_train)

956707

In [113]:
len(labels_train)

956707

In [114]:
array_true_ex_train=[]
for i in range(len(labels_train)):
  if labels_train[i]==1:
    array_true_ex_train.append(examples_train[i])

from collections import defaultdict
import json
def create_user_movie_dict_df2ex(df):

    # if not os.path.isfile('user_movie_train.json'):
    user_movie = defaultdict(dict)
    for iter, row in df.iterrows():
        
        dict1={}
        dict1=user_movie[row[1]].copy()
        dict1[row[0]]=row[2]
        user_movie[row[1]]=dict1
    #     my_json = json.dumps(user_movie)
    #     f = open("user_movie_train.json","w")
    #     f.write(my_json)
    #     f.close()
    # else:
    #     print("opening json file")
    #     with open('user_movie_train.json') as json_file:
    #       user_movie = json.load(json_file)
    # print('user_movie_train.json loaded')
    return user_movie

df2_explicit_dict=create_user_movie_dict_df2ex(df2_explicit)

ratings_train=[]
for i in range(len(array_true_ex_train)):
  movie=array_true_ex_train[i][0]
  user=array_true_ex_train[i][1]
  ratings_train.append(df2_explicit_dict[user][movie])
len(ratings_train)

array_true_ex_select=[]
for i in range(len(labels_model_selection)):
  if labels_model_selection[i]==1:
    array_true_ex_select.append(examples_model_selection[i])

ratings_selection=[]
for i in range(len(array_true_ex_select)):
  movie=array_true_ex_select[i][0]
  user=array_true_ex_select[i][1]
  ratings_selection.append(df2_explicit_dict[user][movie])
len(ratings_selection)






119661

In [115]:
array_true_ex_train[0]

array(['m14296', 'u88118'], dtype=object)

In [116]:
array_true_ex_train

[array(['m14296', 'u88118'], dtype=object),
 array(['m15339', 'u70392'], dtype=object),
 array(['m9028', 'u45538'], dtype=object),
 array(['m4640', 'u31985'], dtype=object),
 array(['m8832', 'u98977'], dtype=object),
 array(['m7617', 'u2807'], dtype=object),
 array(['m937', 'u89994'], dtype=object),
 array(['m14473', 'u92339'], dtype=object),
 array(['m15568', 'u77457'], dtype=object),
 array(['m14712', 'u21722'], dtype=object),
 array(['m443', 'u12525'], dtype=object),
 array(['m13436', 'u25354'], dtype=object),
 array(['m3254', 'u42072'], dtype=object),
 array(['m2152', 'u10897'], dtype=object),
 array(['m15739', 'u9424'], dtype=object),
 array(['m5571', 'u89759'], dtype=object),
 array(['m5239', 'u49803'], dtype=object),
 array(['m12911', 'u27310'], dtype=object),
 array(['m16879', 'u28277'], dtype=object),
 array(['m17355', 'u89855'], dtype=object),
 array(['m5633', 'u34201'], dtype=object),
 array(['m13648', 'u46011'], dtype=object),
 array(['m12292', 'u78456'], dtype=object),
 ar

In [117]:
from collections import defaultdict
import json
def create_user_movie_dict_df2ex(df):

    # if not os.path.isfile('user_movie_train.json'):
    user_movie = defaultdict(dict)
    for iter, row in df.iterrows():
        
        dict1={}
        dict1=user_movie[row[1]].copy()
        dict1[row[0]]=row[2]
        user_movie[row[1]]=dict1
    #     my_json = json.dumps(user_movie)
    #     f = open("user_movie_train.json","w")
    #     f.write(my_json)
    #     f.close()
    # else:
    #     print("opening json file")
    #     with open('user_movie_train.json') as json_file:
    #       user_movie = json.load(json_file)
    # print('user_movie_train.json loaded')
    return user_movie

In [118]:
df2_explicit_dict=create_user_movie_dict_df2ex(df2_explicit)

In [119]:
ratings_train=[]
for i in range(len(array_true_ex_train)):
  movie=array_true_ex_train[i][0]
  user=array_true_ex_train[i][1]
  ratings_train.append(df2_explicit_dict[user][movie])
len(ratings_train)

478281

In [120]:
labels_model_selection[:15]

array([1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1])

In [121]:
len(examples_model_selection)

239177

In [122]:
len(labels_model_selection)

239177

In [123]:
array_true_ex_select=[]
for i in range(len(labels_model_selection)):
  if labels_model_selection[i]==1:
    array_true_ex_select.append(examples_model_selection[i])

In [124]:
len(array_true_ex_select)

119661

In [125]:
ratings_selection=[]
for i in range(len(array_true_ex_select)):
  movie=array_true_ex_select[i][0]
  user=array_true_ex_select[i][1]
  ratings_selection.append(df2_explicit_dict[user][movie])
len(ratings_selection)

119661

In [ ]:
ratings_selection

# Train Node2Vec Explicit 

# L1

In [127]:
clf_l1_ex = train_model_l1_ex(
        array_true_ex_train, ratings_train, embedding_train, operator_l1
    )

Epoch 1/40
9566/9566 [==============================] - 19s 2ms/step - loss: 1.4479
Epoch 2/40
9566/9566 [==============================] - 18s 2ms/step - loss: 1.1416
Epoch 3/40
9566/9566 [==============================] - 19s 2ms/step - loss: 1.0845
Epoch 4/40
9566/9566 [==============================] - 19s 2ms/step - loss: 1.0781
Epoch 5/40
9566/9566 [==============================] - 19s 2ms/step - loss: 1.0748
Epoch 6/40
9566/9566 [==============================] - 19s 2ms/step - loss: 1.0729
Epoch 7/40
9566/9566 [==============================] - 18s 2ms/step - loss: 1.0718
Epoch 8/40
9566/9566 [==============================] - 18s 2ms/step - loss: 1.0713
Epoch 9/40
9566/9566 [==============================] - 19s 2ms/step - loss: 1.0704
Epoch 10/40
9566/9566 [==============================] - 19s 2ms/step - loss: 1.0695
Epoch 11/40
9566/9566 [==============================] - 19s 2ms/step - loss: 1.0694
Epoch 12/40
9566/9566 [==============================] - 20s 2ms/step - lo

In [128]:
link_features_select_ex = link_examples_to_features(
        array_true_ex_select, embedding_train, operator_l1)
predicted_concat_selection_ex = clf_l1_ex.predict(np.array(link_features_select_ex))
predicted_concat_selection_ex=predicted_concat_selection_ex.flatten()
loss_concat_selection_ex=np.sum((ratings_selection-predicted_concat_selection_ex)**2)/len(predicted_concat_selection_ex)
loss_concat_selection_ex
RMSE=np.sqrt(np.sum((np.array(ratings_selection)-predicted_concat_selection_ex)**2)/len(predicted_concat_selection_ex))
RMSE
MAPE=np.sum(np.abs((np.array(ratings_selection)-predicted_concat_selection_ex))/ratings_selection)/len(predicted_concat_selection_ex)
MAPE

3740/3740 [==============================] - 4s 1ms/step


0.32976345612399244

# Test L1 Ex

In [129]:
array_true_ex_test=[]
for i in range(len(labels_test)):
  if labels_test[i]==1:
    array_true_ex_test.append(examples_test[i])

ratings_test=[]
for i in range(len(array_true_ex_test)):
  movie=array_true_ex_test[i][0]
  user=array_true_ex_test[i][1]
  ratings_test.append(df2_explicit_dict[user][movie])
len(ratings_test)

link_features_test_ex = link_examples_to_features(
        array_true_ex_test, embedding_test, operator_l1)
predicted_concat_test_ex = clf_l1_ex.predict(np.array(link_features_test_ex))
predicted_concat_test_ex=predicted_concat_test_ex.flatten()

loss_concat_test_ex=np.sum((ratings_test-predicted_concat_test_ex)**2)/len(predicted_concat_test_ex)
loss_concat_test_ex

RMSE=np.sqrt(np.sum((np.array(ratings_test)-predicted_concat_test_ex)**2)/len(predicted_concat_test_ex))
RMSE

MAPE=np.sum(np.abs((np.array(ratings_test)-predicted_concat_test_ex))/ratings_test)/len(predicted_concat_test_ex)
MAPE

23358/23358 [==============================] - 27s 1ms/step


0.3403822886340566

In [130]:
loss_concat_test_ex

1.148279111827102

In [131]:
RMSE

1.0715778608328477

In [132]:
MAPE

0.3403822886340566

# Concat

In [133]:
clf_concat_ex = train_model_concat_ex(
        array_true_ex_train, ratings_train, embedding_train, operator_concat
    )

Epoch 1/40
9566/9566 [==============================] - 19s 2ms/step - loss: 1.1961
Epoch 2/40
9566/9566 [==============================] - 19s 2ms/step - loss: 1.0232
Epoch 3/40
9566/9566 [==============================] - 20s 2ms/step - loss: 0.9791
Epoch 4/40
9566/9566 [==============================] - 19s 2ms/step - loss: 0.9695
Epoch 5/40
9566/9566 [==============================] - 19s 2ms/step - loss: 0.9635
Epoch 6/40
9566/9566 [==============================] - 19s 2ms/step - loss: 0.9589
Epoch 7/40
9566/9566 [==============================] - 19s 2ms/step - loss: 0.9552
Epoch 8/40
9566/9566 [==============================] - 19s 2ms/step - loss: 0.9517
Epoch 9/40
9566/9566 [==============================] - 19s 2ms/step - loss: 0.9491
Epoch 10/40
9566/9566 [==============================] - 19s 2ms/step - loss: 0.9466
Epoch 11/40
9566/9566 [==============================] - 19s 2ms/step - loss: 0.9445
Epoch 12/40
9566/9566 [==============================] - 19s 2ms/step - lo

In [134]:
clf_concat_ex.save('clf_concat_ex.model')
# from tensorflow import keras
# clf_concat_ex = keras.models.load_model('clf_concat_ex.model')

In [135]:
link_features_select_ex = link_examples_to_features(
        array_true_ex_select, embedding_train, operator_concat)
predicted_concat_selection_ex = clf_concat_ex.predict(np.array(link_features_select_ex))
predicted_concat_selection_ex=predicted_concat_selection_ex.flatten()
loss_concat_selection_ex=np.sum((ratings_selection-predicted_concat_selection_ex)**2)/len(predicted_concat_selection_ex)
loss_concat_selection_ex
RMSE=np.sqrt(np.sum((np.array(ratings_selection)-predicted_concat_selection_ex)**2)/len(predicted_concat_selection_ex))
RMSE
MAPE=np.sum(np.abs((np.array(ratings_selection)-predicted_concat_selection_ex))/ratings_selection)/len(predicted_concat_selection_ex)
MAPE

3740/3740 [==============================] - 4s 1ms/step


0.2974818025646902

In [136]:
RMSE

0.9663831112167076

In [137]:
predicted_concat_selection_ex = clf_concat_ex.predict(np.array(link_features_select_ex))
predicted_concat_selection_ex=predicted_concat_selection_ex.flatten()

3740/3740 [==============================] - 4s 1ms/step


In [138]:
predicted_concat_selection_ex

array([3.8493724, 3.4844003, 3.611464 , ..., 3.867021 , 3.897336 ,
       4.0989695], dtype=float32)

In [139]:
loss_concat_selection_ex=np.sum((ratings_selection-predicted_concat_selection_ex)**2)/len(predicted_concat_selection_ex)
loss_concat_selection_ex

0.9338963176448836

In [140]:
ratings_selection[-4:]

[5, 5, 5, 4]

In [141]:
RMSE=np.sqrt(np.sum((np.array(ratings_selection)-predicted_concat_selection_ex)**2)/len(predicted_concat_selection_ex))
RMSE

0.9663831112167076

In [142]:
MAPE=np.sum(np.abs((np.array(ratings_selection)-predicted_concat_selection_ex))/ratings_selection)/len(predicted_concat_selection_ex)
MAPE

0.2974818025646902

# Test Explicit Node2Vec

In [143]:
len(examples_test)

1494854

In [144]:
len(labels_test)

1494854

In [145]:
array_true_ex_test=[]
for i in range(len(labels_test)):
  if labels_test[i]==1:
    array_true_ex_test.append(examples_test[i])

ratings_test=[]
for i in range(len(array_true_ex_test)):
  movie=array_true_ex_test[i][0]
  user=array_true_ex_test[i][1]
  ratings_test.append(df2_explicit_dict[user][movie])
len(ratings_test)

link_features_test_ex = link_examples_to_features(
        array_true_ex_test, embedding_test, operator_concat)
predicted_concat_test_ex = clf_concat_ex.predict(np.array(link_features_test_ex))
predicted_concat_test_ex=predicted_concat_test_ex.flatten()

loss_concat_test_ex=np.sum((ratings_test-predicted_concat_test_ex)**2)/len(predicted_concat_test_ex)
loss_concat_test_ex

RMSE=np.sqrt(np.sum((np.array(ratings_test)-predicted_concat_test_ex)**2)/len(predicted_concat_test_ex))
RMSE

MAPE=np.sum(np.abs((np.array(ratings_test)-predicted_concat_test_ex))/ratings_test)/len(predicted_concat_test_ex)
MAPE

23358/23358 [==============================] - 28s 1ms/step


0.3468014838950562

In [146]:
loss_concat_test_ex

1.4266383999617176

In [147]:
RMSE

1.1944196917171608

In [148]:
MAPE

0.3468014838950562

In [149]:
len(array_true_ex_test)

747427

In [150]:
ratings_test=[]
for i in range(len(array_true_ex_test)):
  movie=array_true_ex_test[i][0]
  user=array_true_ex_test[i][1]
  ratings_test.append(df2_explicit_dict[user][movie])
len(ratings_test)

747427

In [151]:
link_features_test_ex = link_examples_to_features(
        array_true_ex_test, embedding_test, operator_concat)
predicted_concat_test_ex = clf_concat_ex.predict(np.array(link_features_test_ex))
predicted_concat_test_ex=predicted_concat_test_ex.flatten()

23358/23358 [==============================] - 27s 1ms/step


In [152]:
predicted_concat_test_ex

array([2.3273487, 4.325609 , 4.3286285, ..., 3.8423944, 3.3535705,
       3.9380345], dtype=float32)

In [153]:
loss_concat_test_ex=np.sum((ratings_test-predicted_concat_test_ex)**2)/len(predicted_concat_test_ex)
loss_concat_test_ex

1.4266383999617176

In [154]:
RMSE=np.sqrt(np.sum((np.array(ratings_test)-predicted_concat_test_ex)**2)/len(predicted_concat_test_ex))
RMSE

1.1944196917171608

In [155]:
MAPE=np.sum(np.abs((np.array(ratings_test)-predicted_concat_test_ex))/ratings_test)/len(predicted_concat_test_ex)
MAPE

0.3468014838950562